In [103]:
# pip install librosa

In [104]:
'''
Please install this specific version of resampy for librosa to work without errors.
'''

'\nPlease install this specific version of resampy for librosa to work without errors.\n'

In [105]:
# pip install resampy==0.3.1

In [106]:
import soundfile
import os
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import librosa
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report
import warnings; warnings.filterwarnings('ignore')

In [107]:
emotions ={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

### Data for binary classification


In [108]:
def load_extract_features(data_path):

    '''
    load_extract_features() is a function that is used to load all the audio files one at a time, compute their features and return the features as well as the target values.

    There are around 8-10 audio files which are corrupted. We hardcode zero values for such files in order to maintain consistency.

    ['calm', 'happy'] emotion data is categorized into 'positive' and  ['angry', 'fearful'] into 'negative'

    Returns:
    1. Features
    2. Binary Target Values
    '''
    final_features,target_emotions, binary_label = [],[], []
    count = 0
    
    for i in glob.glob(data_path + "/Actor_*/*.wav"): #Loop to read every file.
        
        name = os.path.basename(i)
        #We split the name of the file to understand the emotion associated with the file.
        split = name.split("-")
        #We know that the third identifier is associated with the emotion of the audio file. Hence, we use [2] as it represents the third identifier.
        emotion = emotions[split[2]]

        #Below is the code to categorize the emotions into two classes to make this a binary problem.
        if emotion in ['calm', 'happy']:
            binary_label.append(0)
        elif emotion in ['angry', 'fearful']:
            binary_label.append(1)
        else:
            continue
        
        with soundfile.SoundFile(i) as audio:
            waveform = audio.read(dtype="float32")
            sr = audio.samplerate
            
            #Below is the code to extract the Mel spectrogram features
            #128 is the standard for machine learning applications using Mel spectrograms
            m_feature = librosa.feature.melspectrogram(y=waveform, sr=sr, n_mels=128, fmax=sr / 2.0).T
            melspectrogram = np.mean(m_feature,axis=0)
            if melspectrogram.shape != (128,):
                melspectrogram = np.zeros(128)
            
            #Below is the code to extract the chromagram features
            stft_wave = librosa.stft(waveform)
            stft = np.abs(stft_wave)
            c_feature = librosa.feature.chroma_stft(S=stft, sr=sr).T
            chromagram = np.mean(c_feature,axis=0)
            
            #12 is the number of pitch classes
            if chromagram.shape != (12,):
                chromagram = np.zeros(12)
                
            features=np.array([])
            features=np.hstack((chromagram, melspectrogram))
        
            final_features.append(features)
            target_emotions.append(emotion)
            
            count += 1
            if count % 100 == 0:
                print("Processed Audio File Number: ", count)
    
    #We return the features and the binary target values.
    return np.array(final_features), np.array(binary_label)

In [109]:
#Please change the path below to the path of the folder saved in your computer.
# data_path = './Audio_Speech_Actors_01-24'
# X, binary_label = load_extract_features(data_path)
# print(X.shape)
# print(binary_label.shape)


In [110]:
# np.savetxt('matrix.txt', X, delimiter = ',')  
# np.savetxt('binary_label.txt',binary_label,delimiter = ',')

In [111]:
X = np.loadtxt(open("matrix.txt","rb"), delimiter=",", skiprows=0)
binary_label = np.loadtxt(open("binary_label.txt","rb"), delimiter=",", skiprows=0)

print(X.shape) #should be (768,140)
print(binary_label.shape) # should be (768,)

(768, 140)
(768,)


In [112]:
X_bia = np.c_[X, np.ones(X.shape[0])]   # Pad 1's for the bias term
l = 1  # lambda

new_label = np.ones((binary_label.shape[0]))

for i in range (binary_label.shape[0]):
    if binary_label[i] == 0:
        new_label[i] = -1

X_train, X_test, y_train, y_test = train_test_split(X_bia, new_label, random_state=0)

In [113]:
def compute_cost(W, X, Y, l):
    # calculate hinge loss
    n_s = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss =  (np.sum(distances) / n_s)
    # calculate cost
    cost = l / 2 * np.dot(W, W) + hinge_loss 
    
    return cost

In [114]:
def calculate_gradient(W, X, Y,l):

    distance = 1 - (Y * np.dot(X, W))
    dw = np.zeros(len(W))

    for ind, d in enumerate(distance):
        if (ind == W.shape[0]):
            if max(0, d) != 0:
                di =  - ( Y[ind] * X[ind])
        else:
            if max(0, d) == 0:
                di = l*W
            else:
                di = l*W - ( Y[ind] * X[ind])
        dw += di

    dw = dw/len(Y)  # average

    return dw

In [115]:
def svm_accuracy(W,X,Y):
    count = 0

    for i in range(X.shape[0]):
        yp = np.sign(np.dot(W, X[i])) #model
        if(yp == Y[i]):
            count = count +1

    return count / Y.shape[0]

In [116]:
def svm(X, Y, step, l):
    w = np.zeros(X_train.shape[1])
    cost = compute_cost(w, X, Y, l)
    for i in range(1000):
        grad = calculate_gradient(w, X, Y, l)
        new_w = w - step * grad
        new_cost = compute_cost(new_w, X, Y, l)
        print(cost)
        if (new_cost<cost):
            w = new_w
            cost = new_cost
        else:
            break
        
    return w

In [117]:
w_result = svm(X_train, y_train, 0.0001, l)
print(svm_accuracy(w_result, X_train, y_train))
# print(svm_accuracy(w_result,X_test,y_test))

1.0
0.998804302555801
0.9976088440315786
0.9964136243796137
0.995218643552196
0.9940239015016255
0.9928293981802112
0.9916351335402719
0.9904411075341358
0.9892473201141407
0.9880537712326344
0.986860460841973
0.9856673888945227
0.9844745553426598
0.9832819601387699
0.9820896032352471
0.9808974845844961
0.9797056041389309
0.9785139618509746
0.9773225576730599
0.9761313915576297
0.9749404634571355
0.9737497733240387
0.9725593211108102
0.9713691067699302
0.9701791302538887
0.9689893915151849
0.9677998905063275
0.9666402173158504
0.965585899040382
0.9646214796825966
0.963657253032166
0.9626932190506006
0.9617293776994192
0.9607657289401473
0.9598022727343187
0.9588390090434749
0.9578759378291648
0.9569130590529449
0.9559503726763795
0.9549878786610408
0.9540255769685091
0.9530634675603711
0.9521015503982216
0.9511524960518978
0.9502617316911754
0.9493711453205501
0.9484807369044714
0.9475905064073972
0.9467004537937908
0.9458105790281238
0.9449208820748752
0.9440313628985298
0.94314202146

In [118]:
def pca(X):
    X_std = np.reshape(np.std(X,axis=0),(X.shape[1],1))
    cov_mat = np.matmul(X_std , X_std.T)
    values, vectors = np.linalg.eig(cov_mat)


In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, binary_label, random_state=0)
pca(X_train)